<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 642 kB 8.6 MB/s 
     |████████████████████████████████| 120 kB 69.3 MB/s 
     |████████████████████████████████| 87 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 55.5 MB/s 
     |████████████████████████████████| 596 kB 72.2 MB/s 
     |████████████████████████████████| 1.1 MB 52.8 MB/s 
     |████████████████████████████████| 6.4 MB 46.7 MB/s 
     |████████████████████████████████| 3.4 MB 39.5 MB/s 
     |████████████████████████████████| 10.9 MB 34.5 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
     |████████████████████████████████| 840 kB 47.9 MB/s 
     |████████████████████████████████| 237 kB 54.0 MB/s 
     |████████████████████████████████| 55.1 MB 50 kB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 48.3 MB 90 kB/s 
     |████████████████████████████████| 99 kB 11.1 MB/s 
     |████████████████████████████████| 352 kB 52.3 MB/s 
     |███████████████

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/drive/MyDrive/traffic_light_dataset/train',
    '/content/drive/MyDrive/traffic_light_dataset/train',
    ['red', 'green', 'yellow']
)

print(len(train_data))

150


In [ ]:
val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/drive/MyDrive/traffic_light_data/test',
    '/content/drive/MyDrive/traffic_light_data/test',
    ['red', 'green', 'yellow']
)
print(len(val_data))

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=64, train_whole_model=True, epochs=92, validation_data=val_data)

Epoch 1/92
2/2 [==============================] - 33s 3s/step - det_loss: 1.8291 - cls_loss: 1.1505 - box_loss: 0.0136 - reg_l2_loss: 0.0632 - loss: 1.8923 - learning_rate: 0.0320 - gradient_norm: 1.0362
Epoch 2/92
2/2 [==============================] - 3s 2s/step - det_loss: 1.7240 - cls_loss: 1.1340 - box_loss: 0.0118 - reg_l2_loss: 0.0632 - loss: 1.7872 - learning_rate: 0.0800 - gradient_norm: 0.8594
Epoch 3/92
2/2 [==============================] - 3s 2s/step - det_loss: 1.5562 - cls_loss: 1.0845 - box_loss: 0.0094 - reg_l2_loss: 0.0632 - loss: 1.6194 - learning_rate: 0.0799 - gradient_norm: 0.8954
Epoch 4/92
2/2 [==============================] - 3s 2s/step - det_loss: 1.3863 - cls_loss: 0.9606 - box_loss: 0.0085 - reg_l2_loss: 0.0632 - loss: 1.4495 - learning_rate: 0.0797 - gradient_norm: 1.1316
Epoch 5/92
2/2 [==============================] - 3s 2s/step - det_loss: 1.2617 - cls_loss: 0.8363 - box_loss: 0.0085 - reg_l2_loss: 0.0632 - loss: 1.3249 - learning_rate: 0.0796 - gradie

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 5s 5s/step



{'AP': 0.6777153,
 'AP50': 0.980198,
 'AP75': 0.82193047,
 'AP_/green': 0.60191214,
 'AP_/red': 0.77837396,
 'AP_/yellow': 0.6528598,
 'APl': -1.0,
 'APm': 0.7621287,
 'APs': 0.5991885,
 'ARl': -1.0,
 'ARm': 0.79641837,
 'ARmax1': 0.6948343,
 'ARmax10': 0.7331384,
 'ARmax100': 0.73674464,
 'ARs': 0.673064}

In [ ]:
model.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5 (KerasLayer)  multiple                  3234464   
                                                                 
 class_net/class-predict (Se  multiple                 2331      
 parableConv2D)                                                  
                                                                 
 box_net/box-predict (Separa  multiple                 2916      
 bleConv2D)                                                      
                                                                 
Total params: 3,239,711
Trainable params: 3,192,575
Non-trainable params: 47,136
_________________________________________________________________


In [ ]:
model.export(export_dir='.', tflite_filename='android.tflite')

In [ ]:
model.evaluate_tflite('android.tflite', val_data)

30/30 [==============================] - 70s 2s/step



{'AP': 0.7494588,
 'AP50': 1.0,
 'AP75': 0.9239845,
 'AP_/green': 0.75488764,
 'AP_/red': 0.7936881,
 'AP_/yellow': 0.69980067,
 'APl': -1.0,
 'APm': 0.7934497,
 'APs': 0.71320236,
 'ARl': -1.0,
 'ARm': 0.8113146,
 'ARmax1': 0.7848928,
 'ARmax10': 0.7848928,
 'ARmax100': 0.7848928,
 'ARs': 0.7537037}

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('android.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''pathrep='/content/drive/MyDrive/Rep-dataset/'
DATAlist = os.listdir(pathrep)

def representative_dataset():
  for i in DATAlist:
    current = cv2.imread(pathrep + i)
    img=cv2.cvtColor(current,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(96, 96))
    img_normalized = cv2.normalize(img, None, -128, 127,
    cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    array = np.array(img_normalized )
    #array = np.expand_dims(array, axis=0)
    array= np.repeat(array[..., np.newaxis], 3, -1)
    array = array.reshape(-1,96,96,3)
    #array = np.expand_dims(array, axis=2)
    #array = ((array / 127.5) - 1.0).astype(np.float32)
    yield([array])'''